In [1]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn as sk
import pickle
import copy
from pyts.classification import TimeSeriesForest

In [8]:
AU_COLS = ['AU01_r',
'AU02_r',
'AU04_r',
'AU05_r',
'AU06_r',
'AU07_r',
'AU09_r',
'AU10_r',
'AU12_r',
'AU14_r',
'AU15_r',
'AU17_r',
'AU20_r',
'AU23_r',
'AU25_r',
'AU26_r',
'AU45_r']

ID_COLS = ['filename', "frame", "video_id", "emotion_1_id"]

In [2]:
load_dotenv()

# path to save figures
output_path = os.getenv("VIDEO_OUT")

In [4]:
load_path = os.path.join(output_path, 'video_data_time_series.csv')
df = pd.read_csv(load_path)

In [5]:
df

,filename,frame,video_id,emotion_1_id,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,...,AU10_r,AU12_r,AU14_r,AU15_r,AU17_r,AU20_r,AU23_r,AU25_r,AU26_r,AU45_r
0,A050121-R_awe_p_3,1,A050121-R,18,0.000,0.000,0.146,0.000000,0.345455,0.174,...,0.250,0.322,0.010,0.000,0.372,0.008,0.046,0.000,0.306,0.058
1,A050121-R_awe_p_3,2,A050121-R,18,0.000,0.000,0.132,0.000000,0.343434,0.192,...,0.254,0.308,0.014,0.000,0.336,0.006,0.046,0.000,0.282,0.026
2,A050121-R_awe_p_3,3,A050121-R,18,0.000,0.000,0.126,0.000000,0.351515,0.206,...,0.264,0.304,0.016,0.000,0.314,0.006,0.040,0.000,0.264,0.008
3,A050121-R_awe_p_3,4,A050121-R,18,0.000,0.000,0.126,0.000000,0.361616,0.222,...,0.266,0.302,0.020,0.000,0.314,0.002,0.034,0.000,0.266,0.046
4,A050121-R_awe_p_3,5,A050121-R,18,0.000,0.000,0.138,0.000000,0.381818,0.274,...,0.266,0.306,0.020,0.008,0.318,0.002,0.022,0.000,0.282,0.134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2238014,A91_sad_p_3,196,A91,6,0.008,0.000,0.000,0.004435,0.000000,0.000,...,0.128,0.142,0.000,0.000,0.000,0.000,0.000,0.328,0.000,0.006
2238015,A91_sad_p_3,197,A91,6,0.000,0.000,0.000,0.000000,0.000000,0.000,...,0.114,0.134,0.000,0.008,0.004,0.000,0.000,0.308,0.000,0.008
2238016,A91_sad_p_3,198,A91,6,0.000,0.002,0.000,0.000000,0.000000,0.000,...,0.100,0.128,0.000,0.020,0.004,0.000,0.000,0.288,0.000,0.004
2238017,A91_sad_p_3,199,A91,6,0.000,0.020,0.000,0.000000,0.000000,0.000,...,0.100,0.128,0.000,0.034,0.010,0.000,0.000,0.276,0.000,0.004


In [17]:
df.frame.min()

1

In [10]:
X_grouped = np.row_stack([
    group.drop(columns=ID_COLS).values[None]
    for _, group in df.groupby('filename')])

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 38 and the array at index 1 has size 69

In [5]:
clf = TimeSeriesForest(random_state=43)
clf.fit(X_train, y_train)

ret = clf.score(X_test, y_test)

In [9]:
from sklearn.model_selection import cross_validate
import numpy as np

In [17]:
scores = cross_validate(X=X_test, y=y_test,
                        estimator           = clf,
                        scoring             = ['accuracy'],
                        verbose             = 1,
                        return_train_score = True
                       )

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    4.0s finished


In [19]:
print('AUC_avg (train):', np.mean(scores['train_roc_auc_ovo_weighted']))
print('AUC_std (train):', np.std(scores['train_roc_auc_ovo_weighted']))
print('AUC_avg (validation):', np.mean(scores['test_roc_auc_ovo_weighted']))
print('AUC_std (validation):', np.std(scores['test_roc_auc_ovo_weighted']))

KeyError: 'train_roc_auc_ovo_weighted'

In [20]:
print('acc_avg (train):', np.mean(scores['train_accuracy']))
print('acc_std (train):', np.std(scores['train_accuracy']))
print('acc_avg (validation):', np.mean(scores['test_accuracy']))
print('acc_std (validation):', np.std(scores['test_accuracy']))

acc_avg (train): 1.0
acc_std (train): 0.0
acc_avg (validation): 0.9666666666666668
acc_std (validation): 0.04216370213557838
